In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

In [3]:
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [4]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [5]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../data')
OUTPUT_DIR = os.path.join(BASE_DIR, '../output')

In [ ]:
# 바꾸지 말것
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to(DEVICE)

In [6]:
# 사이즈 조정은 편하대로 
data = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
dataset_train, dataset_valid = train_test_split(data, test_size=0.3, random_state=SEED)

In [ ]:
print(dataset_train["text"])

724     1B금융o차기 O장 최종Q보I UQ"…&실=)>7임 성공g합
1939               &0L0년이면x-공지능L_건강·P률 상담
2720                 미래 I업 연~소 =험<9 초등학생들
283                    부산국제코미디페스티벌에 참여합니다
805         고용안정지원금 오프라인 신청 첫날…한숨 돌리게 됐어요
                      ...                
2543       박기원 감독 눈치 보지 말고…비예나 눈치 본 건 아닌데
2090      R성 베F남] 갤~6@JsJl개…중x가폰으p 동남아 공H
2649                  아이팩토리 상장폐지 이의신청서 제출
613                LG전자 미국서 G6 사면 구글 홈 준다
1947           미래에셋대우 엔씨소프트 매출 1위 굳건…목표가↑
Name: text, Length: 1960, dtype: object


In [ ]:
import pandas as pd
import re

# 한국어 문자가 아닌 모든 코드 제거 => 유니코드 , 아스키 코드 범위에서 제거
non_korean_pattern = re.compile(r'[^\uac00-\ud7a3]+')

dataset_train['text_cleaned'] = dataset_train['text'].apply(lambda x: non_korean_pattern.sub('[Mask]', str(x)))

print(dataset_train)



                       ID                               text  target  \
724   ynat-v1_train_00724  1B금융o차기 O장 최종Q보I UQ"…&실=)>7임 성공g합       5   
1939  ynat-v1_train_01939             &0L0년이면x-공지능L_건강·P률 상담       4   
2720  ynat-v1_train_02720               미래 I업 연~소 =험<9 초등학생들       3   
283   ynat-v1_train_00283                 부산국제코미디페스티벌에 참여합니다       3   
805   ynat-v1_train_00805      고용안정지원금 오프라인 신청 첫날…한숨 돌리게 됐어요       4   
...                   ...                                ...     ...   
2543  ynat-v1_train_02543     박기원 감독 눈치 보지 말고…비예나 눈치 본 건 아닌데       3   
2090  ynat-v1_train_02090    R성 베F남] 갤~6@JsJl개…중x가폰으p 동남아 공H       4   
2649  ynat-v1_train_02649                아이팩토리 상장폐지 이의신청서 제출       1   
613   ynat-v1_train_00613             LG전자 미국서 G6 사면 구글 홈 준다       4   
1947  ynat-v1_train_01947         미래에셋대우 엔씨소프트 매출 1위 굳건…목표가↑       5   

                        text_cleaned  
724            금융 차기 장 최종 보 실 임 성공 합  
1939                 년이면 공지능 건강 률 상담  
2720              

In [13]:
import pandas as pd
import re

# 데이터 로드
data = pd.read_csv("/data/ephemeral/data/train.csv")

# 각 텍스트에서 ASCII 코드로 변환된 문자들의 아스키 범위 탐색
def get_ascii_info(text):
    ascii_chars = [ord(char) for char in text if ord(char) < 128]
    return ascii_chars

data['ascii_info'] = data['text'].apply(get_ascii_info)

# 아스키 코드 범위 출력
print(data['ascii_info'].head())



0    [105, 32, 58, 49, 32, 122, 32, 75, 84, 40, 32,...
1    [75, 46, 68, 76, 119, 111, 32, 76, 51, 78, 37,...
2           [109, 32, 41, 32, 32, 44, 63, 114, 49, 49]
3                 [56, 32, 32, 50, 55, 32, 32, 32, 32]
4    [112, 73, 73, 32, 82, 50, 102, 114, 32, 93, 32...
Name: ascii_info, dtype: object


In [14]:
# ASCII로 대체된 문자의 위치 찾기
def find_replacement_positions(text):
    return [i for i, char in enumerate(text) if ord(char) < 128]

data['replacement_positions'] = data['text'].apply(find_replacement_positions)
print(data[['text', 'replacement_positions']].head())



                               text  \
0  정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보   
1       K찰.국DLwo 로L3한N% 회장 2 T0&}송=   
2            m 김정) 자주통일 새,?r열1나가야1보   
3     갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩   
4      pI美대선I앞두고 R2fr단 발] $비해 감시 강화   

                               replacement_positions  
0  [1, 2, 3, 5, 6, 9, 10, 11, 12, 13, 14, 19, 20,...  
1  [0, 2, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 18, ...  
2               [0, 1, 4, 5, 10, 12, 13, 14, 16, 20]  
3                    [3, 4, 7, 8, 9, 12, 19, 22, 26]  
4  [0, 1, 5, 9, 10, 11, 12, 13, 15, 17, 18, 19, 2...  


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# N-그램 추출 함수
def extract_n_grams(text, n=2):
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(n, n))
    n_grams = vectorizer.fit_transform([text])
    return vectorizer.get_feature_names_out()

# 각 텍스트에서 2-그램 추출
data['bigrams'] = data['text'].apply(lambda x: extract_n_grams(x, n=2))
data['trigrams'] = data['text'].apply(lambda x: extract_n_grams(x, n=3))

# N-그램 예시 출력
print(data[['text', 'bigrams', 'trigrams']].head())


                               text  \
0  정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보   
1       K찰.국DLwo 로L3한N% 회장 2 T0&}송=   
2            m 김정) 자주통일 새,?r열1나가야1보   
3     갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩   
4      pI美대선I앞두고 R2fr단 발] $비해 감시 강화   

                                             bigrams  \
0  [ 2,  :,  k,  q,  단,  미,  이, ( , 1 , 2e, 2보, :...   
1  [ 2,  t,  로,  회, % , &}, .국, 0&, 2 , 3한, dl, k...   
2  [ 김,  새,  자, ) , ,?, 1나, 1보, ?r, m , r열, 가야, 김...   
3  [ 2,  개,  보,  불,  얼,  주, 27, 7만, 8 , …시, 개통, 갤...   
4  [ $,  r,  감,  강,  발, $비, 2f, ] , fr, i美, i앞, p...   

                                            trigrams  
0  [ 2e,  :파,  kt,  q분,  단],  미사,  이용, ( 이, 1 미, ...  
1  [ 2 ,  t0,  로l,  회장, % 회, &}송, .국d, 0&}, 2 t, ...  
2  [ 김정,  새,,  자주, ) 자, ,?r, 1나가, ?r열, m 김, r열1, ...  
3  [ 27,  개통,  보조,  불법,  얼룩,  주말, 27만, 7만대, 8 주, ...  
4  [ $비,  r2,  감시,  강화,  발], $비해, 2fr, ] $, fr단, ...  


In [21]:
import re

# 한글과 아닌 N-그램 필터링
def filter_korean_ngrams(n_grams):
    non_korean_ngrams = [gram for gram in n_grams if not re.search(r'[가-힣]', gram)]
    return non_korean_ngrams

# 한글이 없는 N-그램 추출
data['non_korean_bigrams'] = data['bigrams'].apply(filter_korean_ngrams)
data['non_korean_trigrams'] = data['trigrams'].apply(filter_korean_ngrams)

# 필터링 결과 출력
print(data[['text', 'non_korean_bigrams', 'non_korean_trigrams']].head())


                               text  \
0  정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보   
1       K찰.국DLwo 로L3한N% 회장 2 T0&}송=   
2            m 김정) 자주통일 새,?r열1나가야1보   
3     갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩   
4      pI美대선I앞두고 R2fr단 발] $비해 감시 강화   

                                  non_korean_bigrams  \
0  [ 2,  :,  k,  q, ( , 1 , 2e, ] , e , i , kt, t...   
1  [ 2,  t, % , &}, 0&, 2 , dl, l3, lw, n%, o , t...   
2                                   [) , ,?, ?r, m ]   
3                                       [ 2, 27, 8 ]   
4                   [ $,  r, 2f, ] , fr, i美, pi, r2]   

                             non_korean_trigrams  
0       [ 2e,  kt, 2e , ] q, i :, kt(, t( , z k]  
1  [ 2 ,  t0, 0&}, 2 t, dlw, lwo, n% , t0&, wo ]  
2                                          [,?r]  
3                                          [ 27]  
4                      [ r2, 2fr, ] $, pi美, r2f]  


In [22]:
from collections import Counter

# 비한글 N-그램 빈도 계산
bigrams = [gram for grams in data['non_korean_bigrams'] for gram in grams]
trigrams = [gram for grams in data['non_korean_trigrams'] for gram in grams]

# 빈도 계산
bigram_counts = Counter(bigrams)
trigram_counts = Counter(trigrams)

# 상위 빈도 N-그램 출력
print("Top non-Korean bigrams:", bigram_counts.most_common(10))
print("Top non-Korean trigrams:", trigram_counts.most_common(10))


Top non-Korean bigrams: [(' 1', 219), (' 2', 164), (' 3', 118), ('t ', 106), (' 4', 94), ('s ', 94), (' 5', 90), (' a', 76), ('c ', 76), (' k', 73)]
Top non-Korean trigrams: [('...', 44), ('kt ', 39), (' 10', 30), (' 20', 22), ('skt', 19), ('mlb', 17), ('5g ', 17), ('lg ', 16), (' ai', 16), (' lg', 15)]
